In [2]:
import sklearn
import numpy as np
import pandas as pd
from ast import literal_eval

In [4]:
data = pd.read_csv('../Data_final/cleaned_data/final_data.csv', converters={'feedbacks': literal_eval})

In [7]:
string_columns = ['name', 'qualification', 'clinic_address', 'current_url', 'feedbacks', 'domain', 'reviews_score_index']

In [11]:
numeric_columns = list(set(data.columns) - set(string_columns))

In [19]:
data_numeric = data[numeric_columns]

In [25]:
X_score = data_numeric.loc[:, data_numeric.columns != 'score']
Y_score = data_numeric.loc[:, data_numeric.columns == 'score']

In [33]:
from sklearn.model_selection import train_test_split
X_score_train, X_score_test, Y_score_train, Y_score_test = train_test_split(X_score, Y_score, test_size=0.1, random_state=42)

In [39]:
def score_model(y_true, y_pred):
    from sklearn.metrics import mean_squared_error, r2_score
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    return {
        'mse': mse,
        'rmse': rmse,
        'r2': r2
    }

In [41]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_score_train, Y_score_train)
pred = model.predict(X_score_test)
score_model(Y_score_test, pred)

{'mse': 0.009888532308961029,
 'rmse': 0.09944109969706202,
 'r2': 0.1173718421378348}

In [47]:
from sklearn.neural_network import MLPRegressor
model = MLPRegressor(hidden_layer_sizes=(8, 4, 2))
model.fit(X_score_train, Y_score_train)
pred = model.predict(X_score_test)
score_model(Y_score_test, pred)

/home/parth20/anaconda3/envs/computer_vision/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'mse': 0.024848241677240673,
 'rmse': 0.15763325054454938,
 'r2': -1.2178981766406491}